#### This code plots the processed Baysor segmentation over an FOV for validation

In [ ]:
import scanpy as sc
import pandas as pd
import numpy as np
import os
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from collections import Counter
import glob
import alphashape
import geopandas as gpd
import seaborn as sns
from shapely.ops import transform
from shapely.affinity import scale
import imageio as io
import tifffile as tiff
import imagecodecs
import shapely.affinity as sa
import json
from core_functions.segmentation_evaluation import *

In [ ]:
data_dir = "D:/amonell/merscope_final"

In [ ]:
input_folders = glob.glob(os.path.join(data_dir, "SI*"))

#### Plotting Segmentation on an FOV

In [ ]:
minx = 500
miny = 500
maxx = 600
maxy = 600

In [ ]:
for input_file in tqdm(input_folders):
    print(input_file)
    adata = sc.read(os.path.join(input_file, "adatas", "01_preprocessed.h5ad"))
    xenium_output_path = os.path.join(input_file, "xenium_output")
    baysor_transcripts = pd.read_csv(
        os.path.join(input_file, "transcripts_assigned.csv"), index_col=0
    )
    print("Finished reading in adata and transcripts")
    pixel_size = 1
    print("Subsetting FOV")
    transcript_subset_fov = subset_transcripts_file(
        baysor_transcripts, pixel_size, minx, maxx, miny, maxy
    )

    print("Making Shapes")
    shapes = (
        transcript_subset_fov[
            (transcript_subset_fov.cell_id != 0) & (transcript_subset_fov.cell_id != -1)
        ]
        .groupby("cell_id")[["x", "y"]]
        .apply(make_alphashape, alpha=0.05)
    )
    shapes = gpd.GeoSeries(shapes)

    def scale_to_image(x, y):
        return (x / pixel_size, y / pixel_size)

    colors = sns.color_palette()[3]
    shapes2 = shapes.apply(lambda x: transform(scale_to_image, x))

    print("Plotting")
    fig, ax = plt.subplots(1, 1, figsize=(15, 15))
    # Create an empty GeoDataFrame to store adjusted polygons
    adjusted_shapes = []

    # Iterate through the shapes DataFrame and adjust each polygon
    for original_polygon in shapes2:
        scaled_polygon = sa.translate(original_polygon, -miny, -minx)
        adjusted_shapes.append(scaled_polygon)

    adjusted_shapes = gpd.GeoSeries(adjusted_shapes)

    # Plot the adjusted polygons
    adjusted_shapes.plot(facecolor=colors, edgecolor="none", alpha=0.2, ax=ax)
    adjusted_shapes.plot(facecolor="none", edgecolor=colors, alpha=0.7, ax=ax)
    # ax.set_xlim((0, 1000))
    # ax.set_ylim((1500, 500))
    # plt.scatter((transcript_subset_fov.x.values/pixel_size) - miny, (transcript_subset_fov.y.values/pixel_size) - minx, s=1, linewidths=0.01, alpha=0.5, c='white')
    try:
        os.mkdir(os.path.join(input_file, "figures"))
    except:
        print("Figures dir already exists")
    try:
        os.mkdir(os.path.join(input_file, "figures", "Segmentation_Evaluation"))
    except:
        print("Segmentation_Evaluation dir already exists")
    plt.savefig(
        os.path.join(
            input_file,
            "figures",
            "Segmentation_Evaluation",
            "baysor_refined_segmentation_fov_example.png",
        )
    )
    plt.show()